In [5]:
import pandas as pd
import random
import pyttsx3
from moviepy.editor import *
from moviepy.video.VideoClip import TextClip
import cv2
import numpy as np

# Function to apply blur using OpenCV
def blur_image(image):
    return cv2.GaussianBlur(image, (25, 51), 0)  # Apply Gaussian blur, adjust kernel size as needed

# Function to convert text to speech and save as an audio file
def text_to_speech(text, filename, rate=200):
    engine = pyttsx3.init()
    engine.setProperty('rate', rate)  # Set the speech rate
    engine.save_to_file(text, filename)  # Save the speech to an audio file
    engine.runAndWait()

# Load questions from Excel
file_path = 'GK.xlsx'
df = pd.read_excel(file_path)

# Randomly choose a background and music
backgrounds = ['bg1.mp4', 'bg2.mp4']  # List of your background video files
music_tracks = ['music1.mp3']  # List of your music tracks

# Loop through each row (question) in the DataFrame
for idx, question_data in df.iterrows():
    background = random.choice(backgrounds)
    music = random.choice(music_tracks)
    
    # Extract data for each question
    year = question_data['Year']
    question = question_data['Question']
    options = [question_data['Option1'], question_data['Option2'], question_data['Option3'], question_data['Option4']]
    answer = question_data['Answer']

    # Create audio clips for the question, options, and answer
    question_audio = f"question_{idx + 1}.mp3"
    text_to_speech(f"Question: {question}", question_audio)

    options_audio = f"options_{idx + 1}.mp3"
    options_text = " ".join([f"Option {i + 1}: {option}" for i, option in enumerate(options)])
    text_to_speech(options_text, options_audio, rate=150)  # Set speech rate to 160 (80% speed)

    answer_audio = f"answer_{idx + 1}.mp3"
    text_to_speech(f"The answer is: {answer}", answer_audio)

    # Create video components
    bg_clip1 = VideoFileClip(background).subclip(0, 20)  # Subclip to 18 seconds for short
    bg_clip = bg_clip1.fl_image(blur_image)  # Apply blur to each frame

    # 1. Year display for 3 seconds with a 3-second timer
    year_clip = TextClip(f"Year: {year}", fontsize=140, color='white').set_position('center').set_duration(3)
    timer_clip_3sec = [TextClip(str(i), fontsize=100, color='red').set_duration(1).set_position(('right', 'top')) for i in range(3, 0, -1)]
    timer_clip_3sec = concatenate_videoclips(timer_clip_3sec)
    year_screen = CompositeVideoClip([bg_clip.subclip(0, 3), year_clip, timer_clip_3sec]).set_duration(3)

    # 2. Question display first for 14 seconds
    question_clip = TextClip(f"Q: {question}", fontsize=100, color='white', size=(900, None), method='caption').set_position(('center', 200)).set_duration(14)
    
    # Centrally align options, wrap text, and space them dynamically
    option_clips = []
    start_y = 700  # Start position for options
    for i, option in enumerate(options, 1):
        option_text = TextClip(f"{i}. {option}", fontsize=80, color='white', size=(900, None), method='caption').set_position(('center', start_y + i * 200)).set_duration(14).set_start(2 + (i - 1) * 3)
        option_clips.append(option_text)
    
    timer_clip_14sec = [TextClip(str(i), fontsize=100, color='red').set_duration(1).set_position(('right', 'top')) for i in range(14, 0, -1)]
    timer_clip_14sec = concatenate_videoclips(timer_clip_14sec)
    question_screen = CompositeVideoClip([bg_clip.subclip(0, 14), question_clip] + option_clips + [timer_clip_14sec]).set_duration(14)

    # 3. Answer display for 5 seconds (new screen, no timer)
    answer_clip = TextClip(f"Answer: {answer}", fontsize=100, color='green', size=(900, None), method='caption').set_position('center').set_duration(3)
    answer_screen = CompositeVideoClip([bg_clip.subclip(0, 5), answer_clip]).set_duration(3)

    # Concatenate all screens
    final_clip = concatenate_videoclips([year_screen, question_screen, answer_screen])

    # Add background music with reduced volume
    audio_clip = AudioFileClip(music).subclip(0, final_clip.duration).volumex(0.2)  # Reduce music volume by 70%

    # Add the question, options, and answer TTS audio to the video with increased volume
    question_audio_clip = AudioFileClip(question_audio).volumex(1.5)  # Increase volume by 50%
    options_audio_clip = AudioFileClip(options_audio).volumex(1.5)   # Increase volume by 50%
    answer_audio_clip = AudioFileClip(answer_audio).volumex(1.5)    # Increase volume by 50%

    # Merge all audio tracks (question, options, answer, and background music)
    final_audio = CompositeAudioClip([audio_clip, question_audio_clip.set_start(3), options_audio_clip.set_start(6), answer_audio_clip.set_start(16)])
    final_clip = final_clip.set_audio(final_audio)

    # Save the final video with a unique filename for each question
    output_filename = f"short_aud{idx + 1}.mp4"  # For example: short_1.mp4, short_2.mp4, etc.
    final_clip.write_videofile(output_filename, fps=24)



Moviepy - Building video short_aud1.mp4.
MoviePy - Writing audio in short_aud1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video short_aud1.mp4



Moviepy - Done !
Moviepy - video ready short_aud1.mp4
